# Project 2 - Forecasting Service Metrics

In [6]:
import numpy as np

In [145]:
import pandas as pd

#Reading the data
X_KV = pd.read_csv('../X_KV.csv')   #JNSM_KV_flashcrowd_1 
Y_KV = pd.read_csv('../Y_KV.csv')   
X_VoD = pd.read_csv('../X_VoD.csv') #JNSM_VoD_flashcrowd_1 
Y_VoD = pd.read_csv('../Y_VoD.csv') 

X_KV_2 = pd.read_csv('../X_KV_2.csv')    #JNSM_KV_flashcrowd_2
Y_KV_2 = pd.read_csv('../Y_KV_2.csv')    
X_VoD_2 = pd.read_csv('../X_VoD_2.csv')  #JNSM_VoD_flashcrowd_2
Y_VoD_2 = pd.read_csv('../Y_VoD_2.csv') 

#Remove the first two columns that index the samples and retrieve all other values by using iloc()
X_KV = X_KV.iloc[:,2:]  
Y_KV = Y_KV.iloc[:, 2:] 
X_VoD = X_VoD.iloc[:,2:]
Y_VoD = Y_VoD.iloc[:, 2:]

X_KV_2 = X_KV_2.iloc[:,2:] 
Y_KV_2 = Y_KV_2.iloc[:, 2:] 
X_VoD_2 = X_VoD_2.iloc[:,2:]
Y_VoD_2 = Y_VoD_2.iloc[:, 2:]

#Concatenate datasets
X_KV = pd.concat([X_KV, X_KV_2], ignore_index=True)
Y_KV = pd.concat([Y_KV, Y_KV_2], ignore_index=True)
X_VoD = pd.concat([X_VoD, X_VoD_2], ignore_index=True)
Y_VoD = pd.concat([Y_VoD, Y_VoD_2], ignore_index=True)

### Task I.4
Data preparation: Use one of the methods described in Project 1 (Advanced), Task 1 to pre-process the trace. Remove possible outliers. Reduce the dimensionality of the feature space to k = 16 using tree-based feature selection. Then, split the processed trace into training and test samples (x(t),y(t)) by assigning the samples with t < T to the training set and t ≥ T to the test set. T is chosen so that the training set contains 70% of the samples.

In [146]:
#1. Pre-processing: column-wise standardization 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_KV_columns_scaled = scaler.fit_transform(X_KV)
X_KV_columns_scaled = pd.DataFrame(X_KV_columns_scaled)

X_VoD_columns_scaled = scaler.fit_transform(X_VoD)
X_VoD_columns_scaled = pd.DataFrame(X_VoD_columns_scaled)

In [150]:
#2. Outlier removal: Threshold that keeps 99% of outliers in the dataset (see Project1 Task IV)

#Number of outliers
#Use this function to find the threshold that keeps 99% of the samples in the dataset
def count(features,thr): 
    counter = 0
    features = features.T
    for col in features:
        if any(np.absolute(features[col]) >= thr):
            counter += 1
    return counter

#For dataset JNSM_KV_flashcrowd 99% of samples correspond to 18.810 samples, i.e. 190 outliers
#Choose threshold 69

#For dataset JNSM_VoD_flashcrowd_1 99% of samples correspond to 34.650 samples, i.e. 350 outliers
#Choose threshold 54

# Function that removes outliers from the features matrix as well as the target scores
# Note: Input is of the format pd.DataFrame
def outlier_detection(features, labels, thr): 
    features = features.T
    for col in features:
        if any(np.absolute(features[col]) >= thr):
            features = features.drop([col],axis=1)
            labels = labels.drop([col],axis=0)
    return features.T, labels #Returns reduced feature matrix and target scores

X_KV = outlier_detection(X_KV_columns_scaled, Y_KV['ReadsAvg'], 69)[0]
Y_KV = outlier_detection(X_KV_columns_scaled, Y_KV['ReadsAvg'], 69)[1]

X_VoD = outlier_detection(X_VoD_columns_scaled, Y_VoD['DispFrames'], 54)[0]
Y_VoD = outlier_detection(X_VoD_columns_scaled, Y_VoD['DispFrames'], 54)[1]

In [151]:
#3. Dimensionality reducton of the feature space

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

#KV service
sfm_KV = SelectFromModel(estimator=RandomForestRegressor(n_estimators=10, max_depth = 5), max_features=16).fit(X_KV, Y_KV)
sfm_KV.estimator_.feature_importances_
feature_index_KV = np.array(range(X_KV.shape[1])) #Features are specified by their position in the design matrix starting from 0

print(f"Top 16 features for JNSM_KV_flashcrowd_1: {feature_index_KV[sfm_KV.get_support()]}") 

#Actual names of the top 16 features
feature_names_KV = [list(X_KV.columns)[i] for i in feature_index_KV[sfm_KV.get_support()]]

#VoD service
#Y_VoD_DispFrames = np.array(Y_VoD['DispFrames']).reshape(-1,1) #Values of "DispFrames"

sfm_VoD = SelectFromModel(estimator=RandomForestRegressor(n_estimators=10, max_depth = 5), max_features=16).fit(X_VoD, Y_VoD)
sfm_VoD.estimator_.feature_importances_
feature_index_VoD = np.array(range(X_VoD.shape[1]))
print(f"Top 16 features for JNSM_VoD_flashcrowd_1: {feature_index_VoD[sfm_VoD.get_support()]}")

#Actual names of the top 16 features
feature_names_VoD = [list(X_VoD.columns)[i] for i in feature_index_VoD[sfm_VoD.get_support()]]

#New design matrices reduced to the top 16 features
X_KV = sfm_KV.transform(X_KV)
X_VoD = sfm_VoD.transform(X_VoD)

Top 16 features for JNSM_KV_flashcrowd_1: [ 135  229  234  235  385  386  603  655  898  902  991  995  996  997
 1717 1718]
Top 16 features for JNSM_VoD_flashcrowd_1: [ 490  614  845  879  883  885  939  968  980 1023 1141 1144 1147 1148
 1149 1356]


In [152]:
from sklearn.model_selection import train_test_split

X_KV_train, X_KV_test, Y_KV_train, Y_KV_test = train_test_split(X_KV, Y_KV, test_size=0.3, shuffle = False)
X_VoD_train, X_VoD_test, Y_VoD_train, Y_VoD_test = train_test_split(X_VoD, Y_VoD, test_size=0.3, shuffle = False)

%store X_KV_train X_KV_test Y_KV_train Y_KV_test
%store X_VoD_train X_VoD_test Y_VoD_train Y_VoD_test

Stored 'X_KV_train' (ndarray)
Stored 'X_KV_test' (ndarray)
Stored 'Y_KV_train' (Series)
Stored 'Y_KV_test' (Series)
Stored 'X_VoD_train' (ndarray)
Stored 'X_VoD_test' (ndarray)
Stored 'Y_VoD_train' (Series)
Stored 'Y_VoD_test' (Series)


### Task I.5
Create a new training set and a new test set with samples of structure ([x(t−l), ..., x(t)], [y(t), ..., y(t+h)]).

In [154]:
#For KV service
def X_KV(samples, l, h):
    if (l>=0 and h>0) or (l>0 and h>=0): 
        matrix = np.empty((samples.shape[0]-l-h,(l+1)*samples.shape[1]))
        for i in range(0, samples.shape[0]-h-l):
            matrix[i] = np.concatenate([samples[j] for j in range(i,i+l+1)])
        return matrix
    if l==0 and h==0: 
        return samples

def Y_KV(targets, l, h):
    targets = np.array(targets)
    
    if (l>=0 and h>0) or (l>0 and h>=0): 
        matrix = np.empty((targets.shape[0]-l-h,h+1))   
        for i in range(0, targets.shape[0]-h-l):
            matrix[i] = np.concatenate([[targets[j]] for j in range(i,i+h+1)])
        return matrix
    if l==0 and h==0: 
        return targets

In [155]:
#For VoD service
def X_VoD(samples, l, h, s):
    if s<=0:
        print('The step size must be >=1.')
        
    if (l>=0 and h>0) or (l>0 and h>=0): 
        matrix = np.empty((samples.shape[0]-s*l-s*h,(l+1)*samples.shape[1]))
        for i in range(0, samples.shape[0]-s*l-s*h):
            matrix[i] = np.concatenate([samples[j] for j in range(i,i+l*s+1,s)])
        return matrix
    if l==0 and h==0: 
        return samples
    
def Y_VoD(targets, l, h, s):
    targets = np.array(targets)
    if s<= 0: 
        print('The step size must be >=1.')
        
    if (l>=0 and h>0) or (l>0 and h>=0): 
        matrix = np.empty((targets.shape[0]-s*l-s*h,h+1))   
        for i in range(0, targets.shape[0]-s*h-s*l):
            matrix[i] = np.concatenate([[targets[j]] for j in range(i,i+s*h+1,s)])
        return matrix
    if l==0 and h==0: 
        return targets

### Task I.7
Using linear regression, train models for l = 0, .., 10 in the feature space and h = 0, .., 10 in the target space. The model with l = 0 corresponds to prediction using the current sample. A model with l > 0 corresponds to learning on l samples into the past and predicting 0, ..., 10 steps into the future. Evaluate the models by computing the error (NMAE) on the test set. Display the results in a table with rows representing the time horizon h = 0, .., 10 and columns representing the lag l = 0, .., 10. You need to train one model for each lag value (11 models in total).

In [156]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [157]:
#KV service

#Function that trains a model given the lag value 
def prediction_KV(l):
    reg = LinearRegression(fit_intercept=True).fit(X_KV(X_KV_train, l, 10),Y_KV(Y_KV_train, l, 10))                                                
    return reg.predict(X_KV(X_KV_test, l, 10))

#Function that outputs the NMAE for a given horizon value
#Outputs array of shape (,11), at each position there is the nmae for the respective lag value
def errors_KV(h):
    if h in range(0,11):
        errors = np.empty(11)
        for l in range(0,11):
            errors[l] = (1/np.mean(Y_KV(Y_KV_test,l,10)[:,:h+1]))*mean_absolute_error(prediction_KV(l)[:,:h+1],Y_KV(Y_KV_test,l,10)[:,:h+1])
        return errors
    else: 
        print('Horizon value needs to be 0,...,10')

In [158]:
#VoD service 

#Choosing s = 30

#Function that trains a model given the lag value
def prediction_VoD(l):
    reg = LinearRegression(fit_intercept=True).fit(X_VoD(X_VoD_train, l, 10, 30),Y_VoD(Y_VoD_train, l, 10, 30))                                                
    return reg.predict(X_VoD(X_VoD_test, l, 10, 30))

#Function that outputs the NMAE for a given horizon value
#Outputs array of shape (,11), at each position there is the nmae for the respective lag value
def errors_VoD(h):
    if h in range(0,11):
        errors = np.empty(11)
        for l in range(0,11):
            errors[l] = (1/np.mean(Y_VoD(Y_VoD_test,l,10,30)[:,:h+1]))*mean_absolute_error(prediction_VoD(l)[:,:h+1],Y_VoD(Y_VoD_test,l,10,30)[:,:h+1])
        return errors
    else: 
        print('Horizon value needs to be 0,...,10')

In [160]:
#Matrices for each service that display the NMAE given h = 0,1,...,10 (rows) and l = 0,1,...,10 (columns)

error_matrix_KV = []
error_matrix_VoD = []
for h in range(0,11):
    error_matrix_KV.append(errors_KV(h))
    error_matrix_VoD.append(errors_VoD(h))

In [177]:
#Obtain Latex code 

import tabulate
from tabulate import tabulate

np.set_printoptions(precision=0)
error_matrix_KV = np.array(error_matrix_KV)*10000
error_matrix_VoD = np.array(error_matrix_VoD)*10000

print(tabulate(error_matrix_KV, tablefmt="latex", floatfmt=".0f"))
print(tabulate(error_matrix_VoD, tablefmt="latex", floatfmt=".0f"))

\begin{tabular}{rrrrrrrrrrr}
\hline
 2158 & 2195 & 2181 & 2184 & 2171 & 2157 & 2155 & 2151 & 2151 & 2138 & 2138 \\
 2144 & 2183 & 2186 & 2179 & 2173 & 2159 & 2153 & 2149 & 2147 & 2140 & 2135 \\
 2128 & 2162 & 2181 & 2181 & 2172 & 2163 & 2154 & 2148 & 2145 & 2139 & 2136 \\
 2123 & 2147 & 2167 & 2179 & 2175 & 2164 & 2158 & 2150 & 2145 & 2139 & 2136 \\
 2138 & 2142 & 2155 & 2168 & 2172 & 2166 & 2158 & 2152 & 2145 & 2139 & 2135 \\
 2165 & 2153 & 2149 & 2157 & 2163 & 2162 & 2158 & 2150 & 2145 & 2138 & 2133 \\
 2197 & 2176 & 2158 & 2152 & 2153 & 2155 & 2154 & 2149 & 2143 & 2137 & 2132 \\
 2226 & 2201 & 2177 & 2159 & 2147 & 2145 & 2146 & 2144 & 2140 & 2134 & 2129 \\
 2259 & 2230 & 2202 & 2177 & 2155 & 2142 & 2139 & 2139 & 2138 & 2133 & 2128 \\
 2288 & 2261 & 2229 & 2200 & 2172 & 2149 & 2136 & 2133 & 2133 & 2130 & 2126 \\
 2316 & 2289 & 2258 & 2226 & 2194 & 2166 & 2144 & 2131 & 2129 & 2127 & 2124 \\
\hline
\end{tabular}
\begin{tabular}{rrrrrrrrrrr}
\hline
 1292 & 1303 & 1294 & 1299 & 1298 & 13